In [ ]:
using SQLite
using StatsPlots; gr() # choose your favourite plotting backend
using PlotMGA
using DataFrames

## Data selection
### We first specify all data that we are intested in, alongside the data columns
### If the data is of a type that needs a different type of processing, we can define the processing as: 
```
function process_fn1(df)
    ...
end
function process_fn2(df)
    ...
end
process(::Val{:new_type}) = process_fn1 ∘ process_fn2
```

In [ ]:
path = "example.sqlite" # sqlite db with data to be plotted

db = SQLite.DB(path)

total_costs = get_param(db, :total_costs, 
    [:mga_iteration, :alternative, :total_value])
storages_invested = get_param(db, :storages_invested, 
    [:mga_iteration, :alternative, :technology, :installation, :total_value])
units_invested = get_param(db, :units_invested, 
    [:mga_iteration, :alternative, :technology, :installation, :total_value])
unit_flows = get_param(db, :unit_flow, 
    [:mga_iteration, :alternative, :technology, :installation, :destination, :total_value])

SQLite.close(db)

In [ ]:
picture_folder = "pics/" # a lot of pictures are generated, so its better to keep them in a folder

## Plotting the flow profiles
### We can plot how the profiles change in every MGA iteration.
### We define all of the commodity flow profile types, along with their properties, that we are inserested in plotting. The profiles are grouped, e.g., according to the commodity of the flow. The group can be defined:
```
[
    ProfileType(profile_name_1, df_filter_predicate_1),
    ProfileType(profile_name_2, df_filter_predicate_2),
    ...
]
```
### The plotting will be done over all of the combinations of the profiles types; in this case: electricity production, electricity storage, heat production, and heat storage
### Note that during plotting, we have to also specify the flow units (MWh or GWh)

In [ ]:
commodity_types = [
    ProfileType("Electricity", :destination=>ByRow(contains("Electricity"))),
    ProfileType("Heat", :destination=>ByRow(contains("Space_Heating")))
]

storages = unique(storages_invested.technology)
production_types = [
    ProfileType("Production", :technology=>ByRow(!in(storages))),
    ProfileType("Storage", :technology=>ByRow(in(storages))),
]

plot_all_mga_profiles(
    unit_flows, PlotMGA.GWh, commodity_types, production_types,
    fig_prefix="$(picture_folder)mga_profile_for_"
)

## Plotting interactions between technologies, and their effects
### We can plot how the capacity of installed technology reacts with different technological installations.
### As the plotting is done over every alternative in database (e.g., used algorithm) we have to define markers to differentiate between them. We have to also define the marker for the optimal solution
### Here interactions are also plotted against the costs, but any df with effects will do (e.g, CO2 emmissions)

In [ ]:
markers = Dict(
    "fuzzy_1h.json" => :x,
    "hsj_05_1h.json" => :circle,
    "hsj_1_1h.json" => :rect,
    "Optimal" => :diamond
)

plot_technology_interactions(
    units_invested, total_costs, markers, 
    fig_prefix="$(picture_folder)technology_interactions_", display_plot=false
)

In [ ]:
plot_technology_effects(
    units_invested, total_costs, 
    fig_prefix="$(picture_folder)technology_effects_on_cost_"
)